# Imports

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from shapely.ops import unary_union
import matplotlib.pyplot as plt
from shapely import wkt

# Constants

In [2]:
CRS = 4326

### Color Pallete

In [3]:
c = {
    'green': '#83bca9',
    'darkgreen': '#3e5641'
}

## Load in USA Parks data

In [5]:
usa_parks = gpd.read_file("../data/USA_Parks/v10/park_dtl.gdb/")

In [6]:
usa_parks = usa_parks.to_crs(epsg=CRS)

# Load US States Boundaries

In [7]:
us_states = pd.read_csv("../data/United_States_Boundary_Files.csv")

In [8]:
us_states['geometry'] = us_states['the_geom'].apply(wkt.loads)
us_states_gdf = gpd.GeoDataFrame(us_states, crs = CRS) 

In [9]:
tn_parks = usa_parks.sjoin(
                     us_states_gdf.loc[us_states_gdf['STUSPS'] == 'TN'],
                     predicate = 'intersects')

In [10]:
tn_parks['name'] = tn_parks['NAME_left']

In [11]:
tn_parks = tn_parks[['name', 'FEATTYPE', 'SQMI', 'geometry']]

In [26]:
# TN Map
# tn_map = folium.Map(tiles = "Cartodb Positron",
#                      location = (36, -86.1),
#                      zoom_start = 7,
#                      prefer_canvas=True)

In [27]:
# add feature groups
# tn_layer = folium.FeatureGroup(name = 'Tennessee Parks', show = True)
# tn_health = folium.FeatureGroup(name = 'Tennessee Health', show = True)

In [28]:
# for i, park in tn_parks.iterrows():
#     textbox = folium.Popup(
#             park['name']
#     )
    
#     folium.GeoJson(
#             park['geometry'],
#             popup = textbox,
#             style_function=lambda x: {"fillColor": c['green'],
#                                                                  'color':c['darkgreen'],
#                                                                 'weight': 0.5}).add_to(tn_layer)
                  

In [29]:
# folium.GeoJson(tn_parks, style_function=lambda x: {"fillColor": c['green'],
#                                                                  'color':c['darkgreen'],
#                                                                 'weight': 0.5}).add_to(tn_map)

In [30]:
# tn_layer.add_to(tn_map)
# folium.LayerControl().add_to(tn_map)

In [25]:
# tn_map

## Load in Places Data

In [13]:
places_df = pd.read_csv('../data/PLACES__Local_Data_for_Better_Health__Place_Data_2023_release_20240504.csv')

In [21]:
# places_tact = pd.read_csv('../data/PLACES__Local_Data_for_Better_Health__Census_Tract_Data_2023_release_20240504.csv')

In [52]:
# data_dict = pd.read_csv('../data/PLACES_and_500_Cities__Data_Dictionary_20240504.csv')

#### Create places geopandas DF 

In [14]:
places_df['geometry'] = places_df['Geolocation'].apply(wkt.loads)
places_gdf = gpd.GeoDataFrame(places_df, crs = CRS) 

## find TN Places data

In [15]:
tn_places = places_gdf.loc[places_gdf['StateAbbr'] == 'TN'][['Year', 'StateAbbr',
                                                             'Category', 'Measure',
                                                             'Data_Value', 'TotalPopulation',
                                                            'geometry', 'LocationID', 'MeasureId']]

####  Find the min distance between each Health data point and the nearest park. Save as CSV

In [16]:
tn_health_loc = tn_places[['LocationID', 'geometry']]
tn_parks_loc = tn_parks[['name', 'geometry']]

In [31]:
# min_dist = []
# for i, r_health in tn_health_loc.iterrows():
#     distances = []
#     for idx, r_park in tn_parks_loc.iterrows():
#         distances.append(r_health['geometry'].distance(r_park['geometry']))
#     min_dist.append(np.min(distances))

In [32]:
# tn_health_loc['distance'] = min_dist

In [33]:
# tn_health_loc.to_csv('../data/TN_health_data_with_distances_to_parks.csv')

#### Measures to Focus on

In [152]:
measures = ['Cancer (excluding skin cancer) among adults aged >=18 years',
           'Cholesterol screening among adults aged >=18 years',
           'Coronary heart disease among adults aged >=18 years',
           'Current asthma among adults aged >=18 years',
           'Current lack of health insurance among adults aged 18-64 years',
           'Depression among adults aged >=18 years',
           'Diagnosed diabetes among adults aged >=18 years',
           'Fair or poor self-rated health status among adults aged >=18 years',
           'High blood pressure among adults aged >=18 years',
           'Obesity among adults aged >=18 years',
           'Stroke among adults aged >=18 years']

#### Filter the DF by measures

#### test

In [ ]:
places_filtered = places_df.loc[places_df['Measure'].isin(measures)]